In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from PIL import Image, ImageFont, ImageDraw
from io import BytesIO
import matplotlib.pyplot as plt

import textwrap
from selenium.webdriver.chrome.options import Options


In [2]:
def break_fix(text, width, font, draw):
    if not text:
        return
    lo = 0
    hi = len(text)
    while lo < hi:
        mid = (lo + hi + 1) // 2
        t = text[:mid]
        w, h = draw.textsize(t, font=font)
        if w <= width:
            lo = mid
        else:
            hi = mid - 1
    t = text[:lo]
    w, h = draw.textsize(t, font=font)
    yield t, w, h
    yield from break_fix(text[lo:], width, font, draw)

def fit_text(img, text, color, font):
    width = img.size[0] - 2
    draw = ImageDraw.Draw(img)
    pieces = list(break_fix(text, width, font, draw))
    height = sum(p[2] for p in pieces)
    if height > img.size[1]:
        raise ValueError("text doesn't fit")
    y = (img.size[1] - height) // 2
    for t, w, h in pieces:
        x = (img.size[0] - w) // 2
        draw.text((x, y), t, font=font, fill=color)
        y += h

In [4]:

driver = webdriver.Chrome()
 
driver.maximize_window()
url = "https://www.randomtextgenerator.com/"
driver.get(url)

font=ImageFont.truetype('/Library/Fonts/Arial Unicode.ttf', 16)
languages = ['pt', 'ru', 'sr', 'es']

for lang in languages:
    for i in range(5000):
        path = 'language_data/' + lang + '/' + str(i) + '.png'
        driver.refresh()

        newsletter = driver.find_element(By.XPATH, value=f"//option[@value='{lang}']")
        newsletter.click()

        driver.find_element(By.XPATH, "//input[@type='submit']").click()

        text = driver.find_element(By.ID, "randomtext_box").text
        text = text.split('\n')[0]
        im = Image.new("RGB", (512, 512), "white")
        fit_text(im, text, (0,0,0), font)

        im.save(path)
    
driver.quit()



In [1]:
!tar cvfz language_data.tar.gz *

a Applications
a Applications/The Jackbox Party Pack 5.app
a Applications/GoToMeeting.app
a Applications/The Jackbox Party Pack 6.app
a Applications/The Jackbox Party Pack 7.app
a Applications/.DS_Store
a Applications/Wineskin
a Applications/.localized
a Applications/Chrome Apps.localized
a Applications/Adobe
a Applications/Drawful 2.app
a Applications/The Jackbox Party Pack 8.app
a Applications/The Jackbox Party Pack 8.app/Contents
a Applications/The Jackbox Party Pack 8.app/Contents/MacOS
a Applications/The Jackbox Party Pack 8.app/Contents/Resources
a Applications/The Jackbox Party Pack 8.app/Contents/Info.plist
a Applications/The Jackbox Party Pack 8.app/Contents/Resources/shortcut.icns
a Applications/The Jackbox Party Pack 8.app/Contents/MacOS/run.sh
a Applications/Drawful 2.app/Contents
a Applications/Drawful 2.app/Contents/MacOS
a Applications/Drawful 2.app/Contents/Resources
a Applications/Drawful 2.app/Contents/Info.plist
a Applications/Drawful 2.app/Contents/Resources/shortcu

a Applications/GoToMeeting.app/Contents/Frameworks/G2MFramework.framework/Versions/A/G2MFramework
a Applications/GoToMeeting.app/Contents/Frameworks/G2MFramework.framework/Versions/A/Resources
a Applications/GoToMeeting.app/Contents/Frameworks/G2MFramework.framework/Versions/A/Resources/Info.plist
a Applications/GoToMeeting.app/Contents/Frameworks/G2MFramework.framework/Versions/A/_CodeSignature/CodeResources
a Applications/GoToMeeting.app/Contents/Library/Audio
a Applications/GoToMeeting.app/Contents/Library/LaunchServices
a Applications/GoToMeeting.app/Contents/Library/LaunchServices/com.logmein.G2MHelperTool
a Applications/GoToMeeting.app/Contents/Library/Audio/Plug-Ins
a Applications/GoToMeeting.app/Contents/Library/Audio/Plug-Ins/HAL
a Applications/GoToMeeting.app/Contents/Library/Audio/Plug-Ins/HAL/GotoAudioDevice.driver
a Applications/GoToMeeting.app/Contents/Library/Audio/Plug-Ins/HAL/GotoAudioDevice.driver/Contents
a Applications/GoToMeeting.app/Contents/Library/Audio/Plug-Ins

a Applications/GoToMeeting.app/Contents/Resources/mockup-audioVoIPSetup.png
a Applications/GoToMeeting.app/Contents/Resources/padlock_unlocked_press.tiff
a Applications/GoToMeeting.app/Contents/Resources/toolbar_everyone.tiff
a Applications/GoToMeeting.app/Contents/Resources/drawing_highlighter_on_viewer_menu.tiff
a Applications/GoToMeeting.app/Contents/Resources/manage_materials_press.tiff
a Applications/GoToMeeting.app/Contents/Resources/g2w_title_bar_logo.tiff
a Applications/GoToMeeting.app/Contents/Resources/recording_paused.tiff
a Applications/GoToMeeting.app/Contents/Resources/drawing_highlighter_cursor.tiff
a Applications/GoToMeeting.app/Contents/Resources/recording_indicator_hover.tiff
a Applications/GoToMeeting.app/Contents/Resources/button_webcam_pause_down.tiff
a Applications/GoToMeeting.app/Contents/Resources/G2M-Logo.pdf
a Applications/GoToMeeting.app/Contents/Resources/grabtab_ss_pause.tiff
a Applications/GoToMeeting.app/Contents/Resources/Red-Meeting-Control.pdf
a Applic

a Applications/GoToMeeting.app/Contents/Resources/Assets.car
a Applications/GoToMeeting.app/Contents/Resources/content_sharing_options_arrow.tiff
a Applications/GoToMeeting.app/Contents/Resources/audio_mode_pstn_unmuted_hover.tiff
a Applications/GoToMeeting.app/Contents/Resources/attendees_mute_all.tiff
a Applications/GoToMeeting.app/Contents/Resources/grabtab_hand_raise.tiff
a Applications/GoToMeeting.app/Contents/Resources/manage_materials.tiff
a Applications/GoToMeeting.app/Contents/Resources/drawing_pen.tiff
a Applications/GoToMeeting.app/Contents/Resources/splitview_divider_grab_vertical.tiff
a Applications/GoToMeeting.app/Contents/Resources/al_voice_phone_self_muted.tiff
a Applications/GoToMeeting.app/Contents/Resources/chevron_down.tiff
a Applications/GoToMeeting.app/Contents/Resources/qa_header_priority.tiff
a Applications/GoToMeeting.app/Contents/Resources/pop_up_button.tiff
a Applications/GoToMeeting.app/Contents/Resources/invite_people_info_pane.tiff
a Applications/GoToMeeti

a Applications/GoToMeeting.app/Contents/Resources/go_to_training_banner.tiff
a Applications/GoToMeeting.app/Contents/Resources/finderbutton.tiff
a Applications/GoToMeeting.app/Contents/Resources/toolbar_webcam.tiff
a Applications/GoToMeeting.app/Contents/Resources/ActiveScreenShare.pdf
a Applications/GoToMeeting.app/Contents/Resources/materials_wmv.tiff
a Applications/GoToMeeting.app/Contents/Resources/echo_on.tiff
a Applications/GoToMeeting.app/Contents/Resources/app_stop_button.tiff
a Applications/GoToMeeting.app/Contents/Resources/al_voice_phone_nopin_muted.tiff
a Applications/GoToMeeting.app/Contents/Resources/grab_tab_divider.tiff
a Applications/GoToMeeting.app/Contents/Resources/microphone.tiff
a Applications/GoToMeeting.app/Contents/Resources/grabtab_alert_locked.tiff
a Applications/GoToMeeting.app/Contents/Resources/audio_pstn_selected.tiff
a Applications/GoToMeeting.app/Contents/Resources/logo_g2w.tiff
a Applications/GoToMeeting.app/Contents/Resources/others_cant_hear_me_on.ti

...

a Applications/GoToMeeting.app/Contents/Resources/es.lproj/Pane_WebinarInfo.strings
a Applications/GoToMeeting.app/Contents/Resources/es.lproj/AskToUnmuteWindow.strings
a Applications/GoToMeeting.app/Contents/Resources/es.lproj/G2MeetingInfoViewController.strings
a Applications/GoToMeeting.app/Contents/Resources/es.lproj/Pane_MeetingInfo.strings
a Applications/GoToMeeting.app/Contents/Resources/es.lproj/G2InProductMessagingViewController.strings
a Applications/GoToMeeting.app/Contents/Resources/es.lproj/G2MeetingLockNotificationViewController.strings
a Applications/GoToMeeting.app/Contents/Resources/es.lproj/G2GrabTabWindowController.strings
a Applications/GoToMeeting.app/Contents/Resources/es.lproj/G2AudioChoiceViewController.strings
a Applications/GoToMeeting.app/Contents/Resources/es.lproj/Pane_Polling.strings
a Applications/GoToMeeting.app/Contents/Resources/en.lproj/G2MStringtable.strings
a Applications/GoToMeeting.app/Contents/Resources/en.lproj/Localizable.strings
a Applications

a Applications/GoToMeeting.app/Contents/Resources/zh-Hans.lproj/G2SettingsPaneWebcamViewController.strings
a Applications/GoToMeeting.app/Contents/Resources/zh-Hans.lproj/G2WebcamRequestViewController.strings
a Applications/GoToMeeting.app/Contents/Resources/zh-Hans.lproj/TabbedAttendeeListViewController.strings
a Applications/GoToMeeting.app/Contents/Resources/zh-Hans.lproj/Pane_WebinarInfo.strings
a Applications/GoToMeeting.app/Contents/Resources/zh-Hans.lproj/AskToUnmuteWindow.strings
a Applications/GoToMeeting.app/Contents/Resources/zh-Hans.lproj/G2MeetingInfoViewController.strings
a Applications/GoToMeeting.app/Contents/Resources/zh-Hans.lproj/Pane_MeetingInfo.strings
a Applications/GoToMeeting.app/Contents/Resources/zh-Hans.lproj/G2InProductMessagingViewController.strings
a Applications/GoToMeeting.app/Contents/Resources/zh-Hans.lproj/G2MeetingLockNotificationViewController.strings
a Applications/GoToMeeting.app/Contents/Resources/zh-Hans.lproj/G2GrabTabWindowController.strings
a

a Applications/GoToMeeting.app/Contents/MacOS/GoToMeeting
a Applications/GoToMeeting.app/Contents/_CodeSignature/CodeResources
a Applications/The Jackbox Party Pack 5.app/Contents
a Applications/The Jackbox Party Pack 5.app/Contents/MacOS
a Applications/The Jackbox Party Pack 5.app/Contents/Resources
a Applications/The Jackbox Party Pack 5.app/Contents/Info.plist
a Applications/The Jackbox Party Pack 5.app/Contents/Resources/shortcut.icns
a Applications/The Jackbox Party Pack 5.app/Contents/MacOS/run.sh
a Creative Cloud Files
a Creative Cloud Files/Icon\r
a Creative Cloud Files/_Cloud documents.webloc
a DataGripProjects
a DataGripProjects/exam
a DataGripProjects/exam/demo-medium-en-20170815.sql
a DataGripProjects/exam/.idea
a DataGripProjects/exam/.idea/sqldialects.xml
a DataGripProjects/exam/.idea/.gitignore
a DataGripProjects/exam/.idea/workspace.xml
a DataGripProjects/exam/.idea/modules.xml
a DataGripProjects/exam/.idea/exam.iml
a Deep Learning HW 1.ipynb
a Deep Learning HW 2.ipynb


a Documents/Adobe/Premiere Pro/14.0/Adobe Premiere Pro Auto-Save
a Documents/Adobe/Premiere Pro/14.0/Adobe Premiere Pro Captured Audio
a Documents/Adobe/Premiere Pro/14.0/Adobe Premiere Pro Captured Audio/Writing Monolgue
a Documents/Adobe/Premiere Pro/14.0/Adobe Premiere Pro Auto-Save/Writing Monolgue-2020-08-02_15-24-51.prproj
a Documents/Adobe/Premiere Pro/14.0/Adobe Premiere Pro Auto-Save/Writing Monolgue.prproj
a Documents/Adobe/Premiere Pro/14.0/Adobe Premiere Pro Captured Video/Writing Monolgue
a Documents/Adobe/Premiere Pro/14.0/Profile-Rahul/Settings
a Documents/Adobe/Premiere Pro/14.0/Profile-Rahul/ArchivedLayouts
a Documents/Adobe/Premiere Pro/14.0/Profile-Rahul/Adobe Premiere Pro Prefs
a Documents/Adobe/Premiere Pro/14.0/Profile-Rahul/LocateDialog Column Settings
a Documents/Adobe/Premiere Pro/14.0/Profile-Rahul/Installed Guides.guides
a Documents/Adobe/Premiere Pro/14.0/Profile-Rahul/SharedView Column Settings
a Documents/Adobe/Premiere Pro/14.0/Profile-Rahul/Mac
a Documen

a Downloads/listings.csv.gz
a Downloads/PS3.Rmd
a Downloads/PS2.pdf
a Downloads/HW6.zip
a Downloads/vtubers will save the world (1).pdf
a Downloads/rs6946_hw5.ipynb
a Downloads/DS_UA_301_Homework_2_Fall_2022.pdf
a Downloads/HW2
a Downloads/gta6_model.ipynb
a Downloads/日本語の作文 (2).pdf
a Downloads/HW5
a Downloads/Linear HW3.pdf
a Downloads/Polished_Proof_3 (2).pdf
a Downloads/.Rhistory
a Downloads/lab1.Rmd
a Downloads/Rahul.l7grammar.pdf
a Downloads/image000007.JPG
a Downloads/pass.pkpass
a Downloads/DD_Silas_Bane_Backstory.docx
a Downloads/rahul_sawhney_rs6946_dlhw2.ipynb
a Downloads/image000006.JPG
a Downloads/Visual Studio Code.app
a Downloads/STAR.csv
a Downloads/lab1.pdf
a Downloads/IMG_7234.HEIC.pdf
a Downloads/ba hw7 q3 runtime.jpg
a Downloads/IMG_2085 (1).HEIC
a Downloads/sakubun 2.pdf
a Downloads/pgadmin-er-diagrams.zip
a Downloads/ScienceDirect_articles_04Feb2022_01-58-30.292
a Downloads/PS3.pdf
a Downloads/PS2.Rmd
a Downloads/DS_UA_301_Homework_3_Fall_2022 (5).pdf
a Downloads/P